In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
% matplotlib inline

# 2.5.2 NY Crime Linear Regression Model Validation

Goal: Use a multivariate linear regression model to predict **property crimes** in New York. Validate using 

Data: https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls

## Data Preparation

#### Training Data (2013)

In [30]:
df = pd.read_csv('nyc_crime.csv', header=4, skipfooter=3, engine='python')

In [31]:
# remove extra column
del df['Unnamed: 13']

# rename columns
df.columns = ['city', 'population', 'violent_crimes', 'murder', 'rape1', 'rape2', 'robbery', 'assault', 'property', 
              'burglary', 'larceny_theft', 'vehicle_theft', 'arson']

# delete rape1 - this column doesn't tell us any information
del df['rape1']

# Remove commas 
column_names = ['city', 'population', 'violent_crimes', 'murder', 'rape2', 'robbery', 'assault', 'property', 
              'burglary', 'larceny_theft', 'vehicle_theft', 'arson']

for column in column_names:
    df[column] = df[column].apply(lambda x: str(x).replace(',', ''))

# Convert to numeric
numeric_columns = ['population', 'violent_crimes', 'murder', 'rape2', 'robbery', 'assault', 'property', 
              'burglary', 'larceny_theft', 'vehicle_theft']

for column in numeric_columns:
    df[column] = pd.to_numeric(df[column])

# Remove arson column.  Too many missing fields to impute data.  Based on footnote in raw excel file, NaN fields represent
# cities where the data reporting wasn't done properly.

del df['arson']

In [32]:
#square population
df['sq_population'] = df['population'] **2

#Encode murder data as categorical
df['murder_mod'] = 0
df.loc[df['murder'] > 0, 'murder_mod'] = 1

#Encode robbery data as categorical
df['robbery_mod'] = 0
df.loc[df['robbery'] > 0, 'robbery_mod'] = 1

#### Test Data (2014)

In [33]:
test = pd.read_csv('nyc_crime_2014.csv', header=4, skipfooter=7, engine='python')

In [34]:
# Apply cleaning script to test dataset

# remove extra column
del test['Unnamed: 13']

# rename columns
test.columns = ['city', 'population', 'violent_crimes', 'murder', 'rape1', 'rape2', 'robbery', 'assault', 'property', 
              'burglary', 'larceny_theft', 'vehicle_theft', 'arson']

# delete unnecessary columns for easier pre-processing
del test['rape1']
del test['rape2']
del test['arson']

In [35]:
# Fill NaN with 0
test.fillna(value=0, inplace=True)

# Remove commas 
column_names = ['city', 'population', 'violent_crimes', 'murder', 'robbery', 'assault', 'property', 
              'burglary', 'larceny_theft', 'vehicle_theft']

for column in column_names:
    test[column] = test[column].apply(lambda x: str(x).replace(',', ''))
    
# Convert to numeric
numeric_columns = ['population', 'murder', 'robbery']

for column in numeric_columns:
    test[column] = pd.to_numeric(test[column])

In [36]:
#square population
test['sq_population'] = test['population'] **2

#Encode murder data as categorical
test['murder_mod'] = 0
test.loc[test['murder'] > 0, 'murder_mod'] = 1

#Encode robbery data as categorical
test['robbery_mod'] = 0
test.loc[test['robbery'] > 0, 'robbery_mod'] = 1

## Create Model
$$ Property crime = \alpha + Population + Population^2 + Murder + Robbery$$

In [37]:
#square population
df['sq_population'] = df['population'] **2

#Encode murder data as categorical
df['murder_mod'] = 0
df.loc[df['murder'] > 0, 'murder_mod'] = 1

#Encode robbery data as categorical
df['robbery_mod'] = 0
df.loc[df['robbery'] > 0, 'robbery_mod'] = 1

### Train

Train the model using 2013 crime data.

In [38]:
# Follow given model for parameter selection
X_train = df[['population', 'sq_population', 'murder_mod', 'robbery_mod']]

In [39]:
# Evaluate correlation between variables. Population and population squared are highly correlated, as expected.
correlation_matrix = X_train.corr()
display(X_train.corr())

,population,sq_population,murder_mod,robbery_mod
population,1.000000,0.998264,0.162309,0.064371
sq_population,0.998264,1.000000,0.133067,0.043983
murder_mod,0.162309,0.133067,1.000000,0.313271
robbery_mod,0.064371,0.043983,0.313271,1.000000


In [40]:
# Instantiate and fit.
regr = linear_model.LinearRegression()
Y_train = df['property']
X_train = df[['population', 'sq_population', 'murder_mod', 'robbery_mod']]
regr.fit(X_train, Y_train)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_train, Y_train))


Coefficients: 
 [ 3.46570268e-02 -2.11108019e-09  1.51866535e+01 -9.62774363e+01]

Intercept: 
 -109.57533562257231

R-squared:
0.9961247104988709


### Validate Model

In [41]:
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression 
from scipy import stats

# Organize data so that variable being predicted and all parameters are in same df
data = df[['property', 'population', 'sq_population', 'murder_mod', 'robbery_mod']]

# Specify interaction
linear_formula = 'property ~ population+sq_population+murder_mod+robbery_mod'

# Run model and print results
lm = smf.ols(formula=linear_formula, data=data).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:               property   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 2.204e+04
Date:                Thu, 08 Mar 2018   Prob (F-statistic):               0.00
Time:                        08:59:05   Log-Likelihood:                -2639.5
No. Observations:                 348   AIC:                             5289.
Df Residuals:                     343   BIC:                             5308.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      -109.5753     40.989     -2.673

The probability of the F-statistic is less than 0.05, indicating a significant F-test and that the model as a whole can explain some of the variance in the outcome.  

Looking at the individual parameters, both murder_mod and robbery_mod have p values > 0.05, indicating there is probably no effect for each feature and dropping them would not adversely affect the R^2.

### Revise Model

In [42]:
# **************Are lines 3-7 necessary to do when revising model or is it taken care of in subsequent lines? 

# Reinstantiate and fit.
regr = linear_model.LinearRegression()
Y_train = df['property']
X_train = df[['population', 'sq_population']]
regr.fit(X_train, Y_train)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_train, Y_train))

# Reorganize data and remove unwanted parameters
data_slim = df[['property', 'population', 'sq_population']]

# Update interaction
linear_formula = 'property ~ population+sq_population'

# Run model and print results
lm_slim = smf.ols(formula=linear_formula, data=data_slim).fit()
print(lm_slim.summary())


Coefficients: 
 [ 3.41379526e-02 -2.04973282e-09]

Intercept: 
 -156.96382064187355

R-squared:
0.9960921521961243
                            OLS Regression Results                            
Dep. Variable:               property   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 4.397e+04
Date:                Thu, 08 Mar 2018   Prob (F-statistic):               0.00
Time:                        08:59:05   Log-Likelihood:                -2640.9
No. Observations:                 348   AIC:                             5288.
Df Residuals:                     345   BIC:                             5299.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
------------

The probability of the F-statistic remains 0 and the R^2 value remains large.  However, the results warm there may be strong multicollinearity in the parameters (which we know is true from the correlation matrix).  This may affect the performance of the model on test data.

### Test Model

Test the model using 2014 crime data.

In [47]:
# ************ Confirm this section is done correctly

# Specify test data
Y_test = test['property']
X_test = test[['population', 'sq_population']]

# Get results
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_test, Y_test))


Coefficients: 
 [ 3.41379526e-02 -2.04973282e-09]

Intercept: 
 -156.96382064187355

R-squared:
0.9938046584128706


Based on the results above, the model performed well with the test data.  
R^2 train = 0.996<br>
R^2 test = 0.993